<a href="https://colab.research.google.com/github/D34dP0oL/4216_Biomedical_DS_and_AI/blob/main/Sheet3/Assignment3_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import numpy as np
import math
import pandas as pd
from scipy.stats import poisson, ttest_ind, shapiro, mannwhitneyu

## Biomedical Data Science & AI

## Assignment 3

#### Group members:  Fabrice Beaumont, Fatemeh Salehi, Genivika Mann, Helia Salimi, Jonah

---
### Exercise 1 - Probability
The amount of wine bottles sold in a shop follows a Poisson distribution with *180*
bottles per week (6 days). If $C$ is the **random variable for bottles per day**, how is:

#### 1.1. The probability that the shop will only sell 20 bottles per day?

The probability distribution of a Poisson random variable $X$ (here amount of sold wine bottles over time) is given by
$$ \mathbb{P}[X=x] = \frac{e^{-\mu} \mu^x}{x!} $$


The given setting gives the following means: $\mu_{\text{week}} = 180 \ \implies \ \mu_{\text{day}} = 180/6 = 30$. 

Thus the probability to sell $20$ bottles per day is:

$\begin{aligned} \mathbb{P}[X=20] &= \frac{e^{-\mu_{\text{day}}} \mu_{\text{day}}^{20}}{20!}\\
&= \frac{e^{-\mu_{\text{day}}} \mu_{\text{day}}^{20}}{20!} \\
&= \frac{e^{-30} 30^{20}}{20!} \qquad
\end{aligned}$


In [71]:
print( f"P[X=20] = {((30**20)*np.e**(-30))/math.factorial(20)}" )

P[X=20] = 0.01341115001283781


#### 1.2. The probability that the demand is more than average for a particular day?

We assume the demand is equal to the number of sold bottles. Since $\mu_{\text{day}}$ is the expected/average amount of sold bottles per day, the probability to sell more than this is:

$\begin{aligned} \mathbb{P}[X> \mu_{\text{day}}] = 1-\mathbb{P}[X\le \mu_{\text{day}}] &= 1-\sum_{i=0}^{\mu_{\text{day}}} \frac{e^{-\mu_{\text{day}}} \mu_{\text{day}}^{i}}{i!}\\
&= 1-\sum_{i=0}^{30} \frac{e^{-30} 30^{i}}{i!}
\end{aligned}$
$$\begin{aligned}
\text{We use cumulative distribution function to compute below term, }\\
\mathbb{P}[X \le \mu_{\text{day}}] =\sum_{i=0}^{30} \frac{e^{-30} 30^{i}}{i!}
\end{aligned}$$

In [72]:
# calculating  P( X <= mu_day ) using cumulative distribution function
from scipy.stats import poisson
print( f"P[X>30] = {1 - poisson.cdf(30, 30)}" )

P[X>30] = 0.45164848742208863


#### 1.3. The expected number of units per day $\mathbb{E}[C]$?

We assume again that units is equal to the number of sold bottles. The expected value (average) was needed and thus computed already in task 1.a:
$$ \mathbb{E}[C] = \mu_{\text{day}} = 30 $$

#### 1.4. What is $\text{Var}[C]$?

In Poisson distribution, the variance equals the mean:
$$ \text{Var}[C] = \mathbb{E}[C] = \mu_{\text{day}} = 30 $$

#### 1.5. The standard deviation of $C$?

It is:
$$ \sigma(C) = \sqrt{\mathbb{E}[C]} = \sqrt{\mu_{\text{day}}} = \sqrt{30} \approx 5.48 $$

In [73]:
np.sqrt(30)

5.477225575051661

---
### Exercise 2 - Hypothesis testing
This exercise illustrates a gene expression data set with its normally distributed values.
Consider the gene expression data of the Golub dataset. Load the file `golub.csv`. 
It contains gene expression data of 3051 genes from 38 tumor mRNA samples. The expression data is organized in a matrix where rows correspond to genes and columns to samples. The tumor class of the columns is given in the file `golub.cl`. The names of the genes (rows) are given in `golub.gnames`.

In [74]:
# load dataset in dataframe and set genes as index column
golub_dataset = pd.read_csv( "golub.csv", index_col=0 )
nr_rows, nr_cols = golub_dataset.shape
print( f"The dataset contains {nr_rows} rows and {nr_cols} columns." )
golub_dataset.head(5)

The dataset contains 3051 rows and 38 columns.


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38
1,-1.45769,-1.39420,-1.42779,-1.40715,-1.42668,-1.21719,-1.37386,-1.36832,-1.47649,-1.21583,-1.28137,-1.03209,-1.36149,-1.39979,0.17628,-1.40095,-1.56783,-1.20466,-1.24482,-1.60767,-1.06221,-1.12665,-1.20963,-1.48332,-1.25268,-1.27619,-1.23051,-1.43337,-1.08902,-1.29865,-1.26183,-1.44434,1.10147,-1.34158,-1.22961,-0.75919,0.84905,-0.66465
2,-0.75161,-1.26278,-0.09052,-0.99596,-1.24245,-0.69242,-1.37386,-0.50803,-1.04533,-0.81257,-1.28137,-1.03209,-0.74005,-0.83161,0.41200,-1.27669,-0.74370,-1.20466,-1.02380,-0.38779,-1.06221,-1.12665,-1.20963,-1.12185,-0.65264,-1.27619,-1.23051,-1.18065,-1.08902,-1.05094,-1.26183,-1.25918,0.97813,-0.79357,-1.22961,-0.71792,0.45127,-0.45804
3,0.45695,-0.09654,0.90325,-0.07194,0.03232,0.09713,-0.11978,0.23381,0.23987,0.44201,-0.39560,-0.62533,0.45181,1.09519,1.09318,0.34300,0.20010,0.38992,0.00641,1.10932,0.21952,-0.72267,0.51690,0.28577,0.61937,0.20085,0.29278,0.26624,-0.43377,-0.10823,-0.29385,0.05067,1.69430,-0.12472,0.04609,0.24347,0.90774,0.46509
4,3.13533,0.21415,2.08754,2.23467,0.93811,2.24089,3.36576,1.97859,2.66468,-1.21583,0.59110,3.26050,-1.36149,0.64180,2.32621,-1.40095,-1.56783,0.83502,-1.24482,-1.60767,-1.06221,3.69445,3.70837,-1.48332,2.36698,-1.27619,2.89604,0.71990,0.29598,-1.29865,2.76869,2.08960,0.70003,0.13854,1.75908,0.06151,1.30297,0.58186
5,2.76569,-1.27045,1.60433,1.53182,1.63728,1.85697,3.01847,1.12853,2.17016,-1.21583,-1.10133,2.59982,-1.36149,0.22853,2.34494,-1.40095,-1.56783,0.94532,-1.24482,-1.60767,-1.06221,3.52458,3.70837,-1.48332,1.79168,-1.27619,2.24892,0.02799,-1.08902,-1.29865,2.00518,1.17454,-1.47218,-1.34158,1.55086,-1.18107,1.01596,0.15788


In [75]:
# load tumor class of columns data, treat tumor class as index
tumor_class = pd.read_csv( 'golub.cl.csv', index_col = 0 )
tumor_class
# ALL tumor class has code 0 and AML tumor class has code 1

,x
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
10,0


In [76]:
# load gene names
gene_names = pd.read_csv( 'golub.gnames.csv', index_col = 0 )
gene_names

,V1,V2,V3
1,36,AFFX-HUMISGF3A/M97935_MA_at (endogenous control),AFFX-HUMISGF3A/M97935_MA_at
2,37,AFFX-HUMISGF3A/M97935_MB_at (endogenous control),AFFX-HUMISGF3A/M97935_MB_at
3,38,AFFX-HUMISGF3A/M97935_3_at (endogenous control),AFFX-HUMISGF3A/M97935_3_at
4,39,AFFX-HUMRGE/M10098_5_at (endogenous control),AFFX-HUMRGE/M10098_5_at
5,40,AFFX-HUMRGE/M10098_M_at (endogenous control),AFFX-HUMRGE/M10098_M_at
...,...,...,...
3047,7121,Breast epithelial antigen BA46 mRNA,U58516_at
3048,7123,TUBULIN ALPHA-4 CHAIN,X06956_at
3049,7125,PTGER3 Prostaglandin E receptor 3 (subtype EP3...,X83863_at
3050,7126,HMG2 High-mobility group (nonhistone chromosom...,Z17240_at


#### 2.1. Calculate the sample mean of all genes $\beta$ in the pooled expression matrix. Use these means to determine the overall mean $\beta_0$ by just taking the average.

In [77]:
# sample mean of all genes
print( 'Sample Mean beta_cap' )
beta_cap = golub_dataset.mean( axis = 1 )
beta_cap

Sample Mean beta_cap


1      -1.129013
2      -0.846746
3       0.260806
4       0.949458
5       0.475348
          ...   
3047    0.422469
3048   -0.353091
3049    0.484367
3050   -0.366183
3051   -0.360164
Length: 3051, dtype: float64

In [78]:
# overall mean of all sample means
beta_0 = golub_dataset.mean( axis = 1 ).mean()
print( 'Sample Mean beta_0:', beta_0 )

Sample Mean beta_0: -6.813986781078161e-09


#### 2.2. Based on the $t$-statistic defined below obtain the $100$ most signifiant genes.
$$ t_{\hat{\beta}} = \frac{ \hat{\beta}-\beta_0}{\text{s.e.}(\hat{\beta})} $$
[Hint: $\hat{\beta}$ is the sample mean of a particular gene.]

In [79]:
# sample standard deviation
sample_std = golub_dataset.std( axis = 1 )
n = golub_dataset.shape[1] # sample_size
degree_of_freedom = n-1
# level of significance, alpha = 0.05

# compute t-statistic for each sample
t_statistic = np.zeros( (beta_cap.shape[0],2), dtype ='int' )
for i in np.arange( start=1, stop = beta_cap.shape[0], step = 1):
    # (i+1) since indices of beta_cap & sample_std start from 1
    t_statistic[i][0]= i #index
    t_statistic[i][1] = ( beta_cap[i+1] - beta_0)/(sample_std[i+1]/(math.sqrt(n))) # t-statistic
test_statistic_dataframe = pd.DataFrame(t_statistic)
test_statistic_dataframe

,0,1
0,0,0
1,1,-9
2,2,3
3,3,3
4,4,1
...,...,...
3046,3046,8
3047,3047,-3
3048,3048,6
3049,3049,-4


In [80]:
# the result is significant if alternative hypothesis is correct
# Comparing t-value to critical value. Checking critical value for t( df = 37, alpha/2=0.025 ) = 2.0262

# 100 most significant genes
selected_genes = test_statistic_dataframe.loc[ abs(test_statistic_dataframe[1])>=2.0262  ]
result = selected_genes.sort_values(by=1, ascending=False) # greater the t-value, more statistically significant the result
np.array(result[:100][0])

array([2029,  271, 2846, 1983, 1013,  747, 2415, 3033, 2696, 2496,  437,
       1144, 1406, 2439,  771, 2330, 2251, 2465,   36, 2477, 3037, 2901,
        124,  824, 3032, 2241, 2504,  968,  485, 1309, 1806, 1733, 1181,
        734,  641, 2829, 1965, 2367, 1058,  569,  105, 1201, 2892, 2277,
        164, 2315,  198,    8, 1740, 1526, 1388, 1620, 2862, 1464, 2848,
       1724, 2962, 2752,  559,  175, 1282, 2495, 2049, 3004, 2651,  252,
       2437,  884, 1754, 1494, 1970, 2446, 1599, 2364,  103, 1722, 2226,
       2915, 2557,   89, 1275, 1887, 1369, 1993, 1812, 2783, 1863, 2835,
       2746, 1024, 2471, 2000,  263, 1820,  127, 1889,  157, 2832, 2094,
       1782])

#### 2.3. Perform two sampled student $t$-tests for all genes comparing the distributions for ALL and AML.

In [81]:
# prepare ALL class dataframe
all_sample_dataframe = golub_dataset.loc[:][golub_dataset.columns[0:27]]

# prepare AML class dataframe
aml_sample_dataframe = golub_dataset.loc[:][golub_dataset.columns[27:]]

# perform two sampled student t-test for each gene

# create array to store t-test results for each gene
t_test_result = np.zeros( (3051, 3) )

for gene in np.arange( start = 1, stop = 3052, step = 1, dtype=int ):
    a = all_sample_dataframe.loc[gene][:]
    b = aml_sample_dataframe.loc[gene][:]
    t_test_result[gene-1][0] = gene # gene index
    ( t_test_result[gene-1][1],  t_test_result[gene-1][2] ) = ttest_ind( a, b) # test result 
t_test_result_dataframe = pd.DataFrame( t_test_result, columns = ['gene', 't-statistic', 'p-value' ] )
t_test_result_dataframe

,gene,t-statistic,p-value
0,1.0,-2.502107,0.017028
1,2.0,-1.156167,0.255228
2,3.0,0.109986,0.913031
3,4.0,0.272658,0.786674
4,5.0,1.186627,0.243147
...,...,...,...
3046,3047.0,-0.042022,0.966714
3047,3048.0,-0.236435,0.814435
3048,3049.0,-1.735692,0.091172
3049,3050.0,-0.093993,0.925636


#### 2.4. Based on the $p$-values obtained in 2.3., obtain the top $10$ genes with the lowest $p$-values.

In [82]:
result = t_test_result_dataframe.sort_values(by= 'p-value', ascending=True)
gene_names.loc[np.array(result[:10]['gene'], dtype = int)]

,V1,V2,V3
829,1882,CST3 Cystatin C (amyloid angiopathy and cerebr...,M27891_at
378,760,CYSTATIN A,D88422_at
2124,4847,Zyxin,X95735_at
808,1834,CD33 CD33 antigen (differentiation antigen),M23197_at
2489,5772,"C-myb gene extracted from Human (c-myb) gene, ...",U22376_cds2_s_at
394,804,Macmarcks,HG1612-HT1612_at
2670,6218,"ELA2 Elastatse 2, neutrophil",M27783_s_at
1009,2288,DF D component of complement (adipsin),M84526_at
1995,4535,RETINOBLASTOMA BINDING PROTEIN P48,X74262_at
937,2121,CTSD Cathepsin D (lysosomal aspartyl protease),M63138_at



#### 2.5. Shapiro-Wilk test is used to test if a random variable follows a Normal distribution (Null-hypothesis). Using this test identify the top $100$ genes which deviate significantly from normal.

In [83]:
# identifying top 100 genes which deviate from normal distribution using Shapiro-Wilks test
# The H_o for Shapiro test is that the variable is normally distributed in some population

# create array to store shapiro-test results for each gene
shapiro_test_result = np.zeros( (3051, 3) )

for gene in np.arange( start = 1, stop = 3052, step = 1, dtype=int ):
    shapiro_test_result[gene-1][0] = gene # gene index
    ( shapiro_test_result[gene-1][1],  shapiro_test_result[gene-1][2] ) = shapiro( golub_dataset.iloc[gene-1] ) # test result 
shapiro_test_result_dataframe = pd.DataFrame( shapiro_test_result, columns = ['gene', 'statistic', 'p-value' ] )

# sorting results based on p-value
result = shapiro_test_result_dataframe.sort_values(by= 'p-value', ascending=True)

# printing top 100 genes
gene_ids = np.array(result[:100]['gene'], dtype = int)
gene_names.loc[gene_ids]

,V1,V2,V3
3022,7066,"PRSS1 Protease, serine, 1 (trypsin 1)",M22612_f_at
2641,6158,Mutant coseg gene for vasopressin-neurophysin ...,X62891_s_at
2551,5930,IG EPSILON CHAIN C REGION,L00022_s_at
459,990,Nkr-P1a Protein,HG4263-HT4533_at
2911,6783,"PDE4C Phosphodiesterase 4C, cAMP-specific (dun...",Z46632_r_at
...,...,...,...
116,260,VLDLR Very low density lipoprotein receptor,D16532_at
2488,5765,KIT V-kit Hardy-Zuckerman 4 feline sarcoma vir...,X06182_s_at
2845,6606,"TCRB T-cell receptor, beta cluster",X00437_s_at
7,42,AFFX-HUMGAPDH/M33197_5_at (endogenous control),AFFX-HUMGAPDH/M33197_5_at


#### 2.6. Out of the $100$ genes obtained in 2.5., use an appropriate statistical test to obtain the $10$ most differentiating genes between ALL and AML classes.

In [84]:
# Applying Mann Whitney Rank test on genes(top 100) which are not normally distributed
# perform mann whitney-test for each gene, H_o of Mann Whitney Rank test assumes the two samples come from same population

# create array to store mann-test results for each gene
mann_test_result = np.zeros( (100, 3) )

# gene_ids contains index/id of genes which are not normally distributed
for i in np.arange(100, dtype=int):
    
    a = all_sample_dataframe.loc[gene_ids[i]][:]
    b = aml_sample_dataframe.loc[gene_ids[i]][:]
    
    mann_test_result[i][0] = gene_ids[i] # gene id
    ( mann_test_result[i][1],  mann_test_result[i][2] ) = mannwhitneyu(a, b) # test result 
    
mann_test_result_dataframe = pd.DataFrame( mann_test_result, columns = ['gene', 'test-statistic', 'p-value' ] )

# 10 most differentiating genes b/w ALL and AML
# sorting results based on p-value
result = mann_test_result_dataframe.sort_values(by= 'p-value', ascending=True)

# printing top 10 most differentiating genes
gene_ids = np.array(result[:10]['gene'], dtype = int)
gene_names.loc[gene_ids]

,V1,V2,V3
1579,3643,GB DEF = 34 kDa mov34 isologue mRNA,U70735_at
2421,5599,Cyclooxygenase-2 (hCox-2) gene,D28235_s_at
2569,5970,"PAI2 Plasminogen activator inhibitor, type II ...",M31551_s_at
2336,5376,Cyclooxygenase-2 (hCox-2) gene,U04636_rna1_at
2738,6355,"G0S2 gene extracted from Human GOS2 gene, 5' f...",M72885_rna1_s_at
1802,4149,THBS1 Thrombospondin 1,X14787_at
1767,4079,"ALDH2 Aldehyde dehydrogenase 2, mitochondrial",X05409_at
2698,6277,Amphiregulin (AR) gene,M30703_s_at
2488,5765,KIT V-kit Hardy-Zuckerman 4 feline sarcoma vir...,X06182_s_at
1307,2997,GB DEF = Selenium-binding protein (hSBP) mRNA,U29091_at


#### 2.7. Inform yourself about the multiple testing problem. Apply one appropriate method to deal with it and explain how it works.

Assume we have a set of hypotheses that we want to test simultaneously. A solution for this is that we can test each hypothesis separately, using some level of significance α. Consider a case where we have 20 hypotheses to test and a significance level of 0.05. The probability of observing at least one significant result just due to chance is: 

$$
P (\text{atleast one significant result}) = 1 − P(\text{no significant results}) = 1 − (1 − 0.05)^{20} ≈ 0.64
$$

So, with 20 tests being considered, we have a 64% chance of observing at least one significant result, even if all of the tests are not significant (most of which will be false alarms). This large number of false alarms produced when you run multiple hypothesis tests is called the multiple testing problem. 
Methods for dealing with multiple testing frequently call for adjusting α in some way so that the probability of observing at least one significant result due to chance remains below our desired significance level.

One method to deal with this problem is **The Bonferroni correction**.
In this method, we set the significance cut-off at α/n. Also, Bonferroni correction is a highly conservative method. It can decrease the rate of false positives, but also extremely increases false negatives.

In [85]:
from statsmodels.stats.multitest import multipletests
alpha = 0.05

print("Test result from Shapiro-Wilk test:\n",shapiro_test_result_dataframe)
print("P-values < 0.05: ", len(shapiro_test_result_dataframe.loc[np.where(shapiro_test_result_dataframe['p-value']<alpha)]))

# Bonferroni correction
# Get Bonferroni corrected P-value

y=multipletests(pvals=shapiro_test_result_dataframe['p-value'], alpha=alpha, method="bonferroni")
print("Bonferroni corrected P-value:\n",y[1])

print("P-values < 0.05: ", len(y[1][np.where(y[1]<alpha)])) 


Test result from Shapiro-Wilk test:
         gene  statistic       p-value
0        1.0   0.589699  4.144559e-09
1        2.0   0.783611  4.930624e-06
2        3.0   0.961777  2.167703e-01
3        4.0   0.924814  1.381648e-02
4        5.0   0.880878  7.677478e-04
...      ...        ...           ...
3046  3047.0   0.981668  7.758895e-01
3047  3048.0   0.880593  7.546362e-04
3048  3049.0   0.945502  6.333631e-02
3049  3050.0   0.981116  7.568102e-01
3050  3051.0   0.875621  5.600382e-04

[3051 rows x 3 columns]
P-values < 0.05:  1549
Bonferroni corrected P-value:
 [1.26450506e-05 1.50433340e-02 1.00000000e+00 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
P-values < 0.05:  404


---
### Exercise 3 - Linear regression
Using the `fish.csv` dataset, generate a multiple linear regression model with `weight` as the response variable and `length1`, `length2`, `length3`, `height`, and `width` as the predictors. Answer the following questions based on the regression model.

In [86]:
# import fish.csv dataset
fish_dataset = pd.read_csv( 'Fish.csv' )
fish_dataset

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...,...
154,Smelt,12.2,11.5,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,11.7,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,12.1,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,13.2,14.3,15.2,2.8728,2.0672


In [87]:
# splitting dataset into response(y) and predictors(X)
y = fish_dataset['Weight']
X = fish_dataset[['Length1', 'Length2', 'Length3', 'Height','Width']]

'''
Generally, the dataset is split into training & test set. The training set is used for training the model
and the test set is used to test the model. We have not split our dataset, since the exercise does not require it.
We can split data into training and test sets, by using Scikit-Learn's built-in train_test_split() method:


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
'''

"\nGenerally, the dataset is split into training & test set. The training set is used for training the model\nand the test set is used to test the model. We have not split our dataset, since the exercise does not require it.\nWe can split data into training and test sets, by using Scikit-Learn's built-in train_test_split() method:\n\n\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)\n"

In [88]:
from sklearn import linear_model
linear_reg_model = linear_model.LinearRegression()
linear_reg_model.fit( X,y )

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#### 3.1. How large is the coefficient of the predictors?

In [89]:
# coefficient of predictors
coefficients = linear_reg_model.coef_
for i in np.arange(len(coefficients)):
    print( f'{X.columns[i]} has coefficient {coefficients[i]}' )

# The largest coefficient is Length1

Length1 has coefficient 62.35521443246453
Length2 has coefficient -6.526752492044295
Length3 has coefficient -29.026218612693484
Height has coefficient 28.297351322276644
Width has coefficient 22.4733066522373


#### 3.2. What is the value of the adjusted $R$-squared? What does this tell us?

In [90]:
# value of adjusted R-squared
n = y.shape[0]
p = len(X.columns)
r_squared = linear_reg_model.score(X,y)
adjusted_r_squared = 1-((1-r_squared**2)*((n-1)/(n-p-1)))
print( f'Adjusted R-squared {adjusted_r_squared}' )

Adjusted R-squared 0.7766649856943888


When we are performing multiple linear regression, the $R^2$ increases as the number of perdictor variables increase regardless of whether these variables improve the model.
This can tackled by calculating *adjusted $R^2$*, which adjusts $R^2$ for the number of predictors(p) relative to the number of datapoints(n) in the model. The adjusted $R^2$ increases with an increase in the no. of predictors only if the added predictors improve the model. The adjusted $R^2$ is given by,
$$R_{adjusted}^{2}= 1-(1-R^2)\frac{n-1}{n-p-1}$$


The adjusted $R^2$ of 0.777 indicates that around 77.7% of the variance in 'weight' can be explained by the model. Around 23% if variance is unexplained by the model as it could be due to unobserved predictors.

#### 3.3. Which predictors lead to the increase in the weight of the fish, and which have a negative effect?

We can determine the association between predictors and response variable by examining the predictor coefficients. The predictors which have a postive coefficient will result in increase in weight of fish, while those with a negative coefficient will result in decrease in weight.


**Predictors leading to increase in weight of fish -**
- Length1 - A unit increase in this predictor will increase the weight of fish by 62.355 units
- Height - A unit increase in this predictor will increase the weight of fish by 28.297 units
- Width -  A unit increase in this predictor will increase the weight of fish by 22.47 units
    
**Predictors leading to decrease in weight of fish -**
- Length2 - A unit increase in this predictor will decrease the weight of fish by 6.527 units
- Length3 - A unit increase in this predictor will decrease the weight of fish by 29.026 units


#### 3.4. Using a simple regression model, predict the weight of a fish with `length1` of $15$?

In [91]:
simple_linear_model = linear_model.LinearRegression()
X_train = X.loc[:][['Length1']]
simple_linear_model.fit( X_train,y )

X_new = pd.DataFrame({'Length1': [15]})
y_predicted = simple_linear_model.predict(X_new)
print( f'Predicted weight of fish {y_predicted}' )

Predicted weight of fish [29.50737928]


#### 3.5. Predict the weight of a fish with `length1`, `height`, and `width` of $20.0$, $7.3$, and $5.3$ using a multiple linear regression model.

In [92]:
multiple_linear_reg_model = linear_model.LinearRegression()
X_train = X.loc[:][ ['Length1', 'Height', 'Width'] ]
multiple_linear_reg_model.fit( X_train,y )

X_new = pd.DataFrame({'Length1': [20.0], 'Height': [7.3], 'Width': [5.3]})
y_predicted = multiple_linear_reg_model.predict( X_new )
print( f'Predicted weight of fish {y_predicted}' )

Predicted weight of fish [272.67135424]


#### 3.6. What are the associated 95% confidence intervals for the predictors, as well as for intercept?

In [93]:
# Associated 95% confidence intervals for predictors as well as for intercept (**For model in 3.e**)

import statsmodels.formula.api as sm

model = sm.ols(formula='Weight ~ Length1 + Height + Width', data= fish_dataset).fit()

# 95% confidence intervals for model with predictors - Length1, Height, Width
print( '95% confidence intervals' )
print( model.conf_int() )

95% confidence intervals
                    0           1
Intercept -570.737665 -457.946453
Length1     18.690241   26.675564
Height       6.056703   21.283255
Width       13.751582   74.385971


In [94]:
# 95% confidence intervals for model with predictors - Length1, Length2, Length3, Height, Width
model2 = sm.ols(formula='Weight ~ Length1 + Length2 + Length3 + Height + Width', data= fish_dataset).fit()

# 95% confidence intervals for model with predictors - Length1, Length2, Length3, Height, Width
print( '95% confidence intervals' )
print( model2.conf_int() )

95% confidence intervals
                    0           1
Intercept -558.009586 -441.164325
Length1    -17.080780  141.791209
Length2    -89.024956   75.971451
Length3    -63.308554    5.256116
Height      11.051977   45.542726
Width      -17.772891   62.719505
